In [83]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from midiutil import MIDIFile

import random

In [ ]:
## Constraints
# 1. # of pianists, 2 hands, 5 fingers
# 2. time apart
# 3. spatial apart (in each hand cannot exceed one octave)
# 4. within one octave > same intensity > notes start at the same time > last for same period > not adjacent

In [2]:
## Parameters
GroupNum = 21 * 2 #21 pianists, 2 hands each
max_note_per_group = 5 #5 fingers

#min_pitch_gap = not set
win_length = 50e-3
max_time_gap = 0.3
max_frame_gap = int(round(max_time_gap / win_length))

6


In [13]:
key_time = 50e-3*0.5
Fs = 48000

keys = scipy.io.loadmat('MATLAB_data/31Jan_jc.mat')['key']
notes_list = scipy.io.loadmat('notes_88.mat')['notes']
freq_list = scipy.io.loadmat('freq_480.mat')['f_cropped']
#freq_list = scipy.io.loadmat('freq_441.mat')['f_cropped']
notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

def get_intensity(cur_key,cur_freq,wave_freq):
    if cur_key == 1:
        sigma = notes_list[1]-notes_list[0]
    else:
        sigma = notes_list[cur_key-1]-notes_list[cur_key-2]
    sigma = sigma/2
    intensity = np.exp(-0.5 *((cur_freq-wave_freq)/sigma)**2)
    #print('cur', cur_freq,'wave',wave_freq)
    return intensity

def picknotes(cur_key,cur_freq):
    j = 0
    while j < len(cur_key):
        if cur_key[j] != 0:
            
            d = {}
            d['note'] = cur_key[j]
            d['start'] = j
            while cur_key[j] != 0 and j < len(cur_key)-1:
                j += 1
            d['end'] = j - 1
            intensity = get_intensity(cur_key[j-1],cur_freq,notes_list[cur_key[j-1]-1])
            d['intensity'] = intensity
            notes.append(d)
        j += 1

notes = []
for i in range(np.shape(keys)[0]):
    cur_key = keys[i]
    cur_freq = freq_list[i]
    picknotes(cur_key,cur_freq)

notes = sorted(notes, key = lambda notes: notes["start"])

def remove_repetitive(notes):
    note_set = [0]*88
    i = 0
    while i < len(notes):
        note = notes[i]
        if note_set[note['note']-1] == 0:
            note_set[note['note']-1] = [(note['start'],note['end'])]
        else:
            if (note['start'],note['end']) in note_set[note['note']-1]:
                del notes[i]
                i -= 1
            else:
                note_set[note['note']-1].append((note['start'],note['end']))
        i += 1
    return notes

new_notes = remove_repetitive(notes)

In [ ]:
#Separate notes into C major and Gb major
def separate_majors(notes):
    C_major = [1,3,4,6,8,9,11]
    Gb_major = [0,2,3,5,7,9,10]

    C_notes = []
    Gb_notes = []

    for i,note in enumerate(notes):
        seq = note['note'] % 12
        if (seq in C_major) and (seq not in Gb_major):
            C_notes.append(notes.pop(i))
        elif (seq in Gb_major) and (seq not in C_major):
            Gb_notes.append(notes.pop(i))

    for i,note in enumerate(notes):
        if i % 2 == 0:
            C_notes.append(notes.pop(i))
        else:
            Gb_notes.append(notes.pop(i))
    return [C_notes, Gb_notes]

In [ ]:
def group_notes(notes_list):
    ### change [{'note':pitch_num, 'start':start_time,'end':end_time}] to 
    return

In [4]:
word_gap = 15
def separate_word(notes):
    word_list = []
    ptr = 0
    for i in range(len(notes)-1):
        if notes[i+1]['start'] - notes[i]['end'] >= word_gap:
            word_list.append(notes[ptr:i+1])
            ptr = i + 1
        if i == len(notes) - 2:
            word_list.append(notes[ptr:])
    return word_list
    

In [ ]:
separate = False



In [18]:
def print_list(l):
    for i in l:
        print(i)

In [5]:
def add_note(pitch_group,note,current_group,notes_group):
    pitch_group.append(note['note'])
    current_group.append(note)
    notes_group.remove(note)
    return pitch_group,current_group,notes_group

In [78]:
def allocation(notes_group):
    allocated_notes=[]
    group = 1
    overlapped = False #true if last group was left hand, current group is right hand, they should not overlap
    while notes_group != [] and group <= GroupNum:
        current_group = []
        if not overlapped:
            pitch_group = [notes_group[0]['note']]
            current_group.append(notes_group.pop(0))
        else:
            for i in range(len(notes_group)):
                if notes_group[i]['note'] > (high + 1):
                    pitch_group = [notes_group[i]['note']]
                    current_group.append(notes_group.pop(i))
                    break
        for current_allocated_note in current_group:
            print('current_group',current_group)
            print(pitch_group)
            candidate = []
            # counter = 0
            for note in notes_group:
                if (note['note'] not in pitch_group) and (note['note']-min(pitch_group) <= 12):
                    #print(note['note'])
                    if overlapped and note['note'] < (high + 1):
                        break
                    else:
                        #print('in')
                        if note['start'] == current_allocated_note['start']:
                            if note['end'] == current_allocated_note['end']:
                                candidate.append([1,note])
                                # counter += 1
                            else:
                                candidate.append([2,note])
                                # counter += 1
                        elif note['end'] == current_allocated_note['end']:
                            candidate.append([3,note])
                        #     counter += 1
                        # elif counter == 4:
                        #     continue
                        else:    
                            candidate.append([4,note])
            print_list(candidate)
        candidate.sort(key = lambda x: x[0])
        print('candidate')
        print_list(candidate)
        counter = 0
        while len(current_group) < 5 and counter < len(candidate):
            if candidate[counter][1]['note'] not in pitch_group:
                pitch_group,current_group,notes_group = add_note(pitch_group,candidate[counter][1],current_group,notes_group)
            counter += 1
                                
        print(current_group)

        if not overlapped:
            high = 0
            for note in current_group:
                high = max(high,note['note'])

        allocated_notes.append(current_group)

        overlapped = not overlapped
        group += 1 #finish allocate notes to one group, update
    
    if notes_group != []:
        print('Failed to allocate :(')
        print(len(notes_group))
        return False
    else:
        return allocated_notes

In [ ]:
if separate:
    ### TOBE FINISHED
    notes_groups = separate_majors(notes)
    result = []
    # for group in notes_groups:
    #     word_groups = separate_word(group)
    #     r = allocation(group)
    #     if not r:
    #         result.append(r)
    #     else:
    #         break
else:
    new_notes = sorted(new_notes, key = lambda notes: notes["start"])
    word_groups = separate_word(new_notes)
    #result = allocation(notes)

In [ ]:
adjacent_note_gap = 0

In [ ]:
## visualization tool
def plot_notes(note_list):
    time =[]
    pitch = []
    for note in note_list:
        time.append(note['start'])
        pitch.append(note['note'])
    plt.scatter(time,pitch,s=1)

In [ ]:
plot_notes(new_notes)
plt.show()
word_groups = separate_word(new_notes)
for g in word_groups:
    plot_notes(g)
plt.show()

In [15]:
for node in word_groups[0]:
    print(node)
print(len(word_groups[0]))

{'note': 35, 'start': 56, 'end': 57, 'intensity': 0.8441018850726608}
{'note': 35, 'start': 56, 'end': 60, 'intensity': 0.5952809114957551}
{'note': 36, 'start': 56, 'end': 61, 'intensity': 0.82167013228532}
{'note': 36, 'start': 56, 'end': 62, 'intensity': 0.9771714362900952}
{'note': 36, 'start': 56, 'end': 63, 'intensity': 0.8308410340862667}
{'note': 36, 'start': 56, 'end': 64, 'intensity': 0.5940073907721798}
{'note': 37, 'start': 56, 'end': 64, 'intensity': 0.8106775193961244}
{'note': 37, 'start': 56, 'end': 65, 'intensity': 0.9669760124800155}
{'note': 37, 'start': 56, 'end': 66, 'intensity': 0.6627387047293724}
{'note': 38, 'start': 56, 'end': 66, 'intensity': 0.7394760143261037}
{'note': 39, 'start': 56, 'end': 66, 'intensity': 0.6055008811396302}
{'note': 39, 'start': 56, 'end': 68, 'intensity': 0.929064605748772}
{'note': 40, 'start': 56, 'end': 68, 'intensity': 0.6161451804438065}
{'note': 40, 'start': 56, 'end': 56, 'intensity': 0.8057397363734694}
{'note': 48, 'start': 5

In [93]:
word_groups = separate_word(new_notes)
test_group = word_groups[1].copy()
result = False
while not result:
    result = allocation(test_group)
    test_group = word_groups[1].copy()
    random.shuffle(test_group)

current_group [{'note': 35, 'start': 100, 'end': 100, 'intensity': 0.9894184139005469}]
[35]
[2, {'note': 36, 'start': 100, 'end': 111, 'intensity': 0.82167013228532}]
[2, {'note': 37, 'start': 100, 'end': 111, 'intensity': 0.8106775193961244}]
[2, {'note': 38, 'start': 100, 'end': 111, 'intensity': 0.7394760143261037}]
[2, {'note': 38, 'start': 100, 'end': 105, 'intensity': 0.787758230195887}]
[2, {'note': 39, 'start': 100, 'end': 105, 'intensity': 0.6055008811396302}]
[2, {'note': 39, 'start': 100, 'end': 104, 'intensity': 0.9533677152955733}]
[2, {'note': 39, 'start': 100, 'end': 103, 'intensity': 0.929064605748772}]
[2, {'note': 39, 'start': 100, 'end': 102, 'intensity': 0.7661319642142077}]
[2, {'note': 40, 'start': 100, 'end': 102, 'intensity': 0.6161451804438065}]
[2, {'note': 40, 'start': 100, 'end': 101, 'intensity': 0.8057397363734694}]
[1, {'note': 42, 'start': 100, 'end': 100, 'intensity': 0.9827976202365659}]
[1, {'note': 43, 'start': 100, 'end': 100, 'intensity': 0.606817

C:\Users\96326\AppData\Local\Temp\ipykernel_29556\668996808.py:22: RuntimeWarning: overflow encountered in ubyte_scalars
  if (note['note'] not in pitch_group) and (note['note']-min(pitch_group) <= 12):


In [88]:
new_notes = sorted(new_notes, key = lambda notes: notes["start"])
word_groups = separate_word(new_notes)
print(word_groups[1])
print(random.shuffle)

[{'note': 35, 'start': 100, 'end': 100, 'intensity': 0.9894184139005469}, {'note': 35, 'start': 100, 'end': 111, 'intensity': 0.5952809114957551}, {'note': 36, 'start': 100, 'end': 111, 'intensity': 0.82167013228532}, {'note': 37, 'start': 100, 'end': 111, 'intensity': 0.8106775193961244}, {'note': 38, 'start': 100, 'end': 111, 'intensity': 0.7394760143261037}, {'note': 38, 'start': 100, 'end': 105, 'intensity': 0.787758230195887}, {'note': 39, 'start': 100, 'end': 105, 'intensity': 0.6055008811396302}, {'note': 39, 'start': 100, 'end': 104, 'intensity': 0.9533677152955733}, {'note': 39, 'start': 100, 'end': 103, 'intensity': 0.929064605748772}, {'note': 39, 'start': 100, 'end': 102, 'intensity': 0.7661319642142077}, {'note': 40, 'start': 100, 'end': 102, 'intensity': 0.6161451804438065}, {'note': 40, 'start': 100, 'end': 101, 'intensity': 0.8057397363734694}, {'note': 42, 'start': 100, 'end': 100, 'intensity': 0.9827976202365659}, {'note': 43, 'start': 100, 'end': 100, 'intensity': 0.

In [94]:
l = 0
for i in result:
    l += len(i)
    print(i)
print(l == len(word_groups[1]))

[{'note': 49, 'start': 100, 'end': 111, 'intensity': 0.7013799383805369}, {'note': 50, 'start': 100, 'end': 111, 'intensity': 0.6305401509327592}, {'note': 52, 'start': 100, 'end': 102, 'intensity': 0.7181438924045347}, {'note': 51, 'start': 100, 'end': 101, 'intensity': 0.9908104469569424}, {'note': 54, 'start': 100, 'end': 101, 'intensity': 0.619504134234769}]
[{'note': 83, 'start': 102, 'end': 102, 'intensity': 0.997951008122305}]
[{'note': 50, 'start': 100, 'end': 104, 'intensity': 0.999071848514152}, {'note': 52, 'start': 100, 'end': 103, 'intensity': 0.9999939375693727}, {'note': 53, 'start': 100, 'end': 102, 'intensity': 0.9878279845418128}, {'note': 55, 'start': 100, 'end': 100, 'intensity': 0.6068171573270484}, {'note': 54, 'start': 100, 'end': 100, 'intensity': 0.8635032255238256}]
[{'note': 77, 'start': 101, 'end': 103, 'intensity': 0.9730755018335165}, {'note': 78, 'start': 101, 'end': 101, 'intensity': 0.7068475662769266}]
[{'note': 40, 'start': 101, 'end': 101, 'intensity

In [ ]:
for a in new_notes:
    print(a)

In [ ]:
note1 = {'note': 28, 'start': 215, 'end': 215, 'intensity': 0.9468900491331496}
note2 ={'note': 29, 'start': 215, 'end': 215, 'intensity': 0.8993647195975836}
note3 ={'note': 29, 'start': 244, 'end': 244, 'intensity': 0.9878279845418128}
l = [[2,note1],[2,note2],[5,note3]]
l.sort(key = lambda x: x[0])
print(l)

In [ ]:
        # candidate = []
        # for i,note in enumerate(notes_group):
        #     if (note['note'] not in pitch_group) and (note['note']-pitch_group[0] <= 12):
        #         if overlapped and note['note'] > (high + 1):
        #             candidate.append(note)
        #         elif not overlapped:
        #             candidate.append(note)
        # for c in candidate:
        #     print(c)
        # for j,note in enumerate(candidate):
        #     if len(current_group) == 5:
        #         break
        #     for current_allocated_note in current_group:
        #         if note['start'] == current_allocated_note['start'] and note['end'] == current_allocated_note['end']:
        #             current_group.append(candidate.pop(j))
        #             notes_group.remove(note)
            
        # for j,note in enumerate(candidate):
        #     if len(current_group) == 5:
        #         break
        #     for current_allocated_note in current_group:
        #         if note['start'] == current_allocated_note['start']:
        #             current_group.append(candidate.pop(j))
        #             notes_group.remove(note)
            
        # for j,note in enumerate(candidate):
        #     if len(current_group) == 5:
        #         break
        #     for current_allocated_note in current_group:
        #         if note['end'] == current_allocated_note['end']:
        #             current_group.append(candidate.pop(j))
        #             notes_group.remove(note)

        # if len(current_group) != 5:
        #     for k in range(5-len(current_group)):
        #         current_group.append(candidate.pop(k))
        #         notes_group.remove(candidate[k])